In [1]:
from glob import glob
import json
import pandas as pd
import plotly.express as px

In [2]:
out = 'out/20220725014352/'

In [3]:
cols = ['order','obs','min','mean','stddev']
df = pd.DataFrame(columns=cols)
for fname in glob(out+'alg1_*.json'):
    with open(fname, 'r') as f:
        d = json.load(f)['results'][0]
    d['obs'] = len(d['times'])
    d['order'] = fname.split('_')[1].split('.')[0]
    d = {k:v for k,v in d.items() if k in cols}
    df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)
df['ci'] = 1.960 * df['stddev'] / (df['obs']**0.5)
col_ref = 'mean'
df = df.sort_values(col_ref)
df['speedup'] = df[df['order']=='ijk'][col_ref][0]/df[col_ref]
df.to_markdown(out+'alg1_order.md', index=False)
df

,order,obs,min,mean,stddev,ci,speedup
3,ikj,10,0.760213,0.777004,0.020941,0.01298,9.357535
2,kij,10,0.812555,0.872689,0.029911,0.018539,8.331541
0,ijk,10,7.247203,7.270846,0.034401,0.021322,1.0
4,jik,10,7.272183,7.319613,0.047343,0.029343,0.993337
1,jki,10,16.210173,16.357423,0.088935,0.055122,0.444498
5,kji,10,16.324115,16.683845,0.355452,0.220311,0.435802


In [4]:
fig = px.bar(df, x='order', y='speedup', text_auto=True)
fig.update_traces(textposition='outside')
fig.update_layout(yaxis_tickformat='.2f',
    margin=dict(l=0, r=0, t=0, b=0))
fig.write_image(out+f'order.png', scale=2)
fig.show()

In [5]:
ref = None

def prep(fname: str, col_ref='mean', filter_ref=True):
    global ref

    with open(fname, 'r') as f:
        d = json.load(f)

    def fn(x):
        y = r['parameters']['blk'].split(',')[x].split('=')[1]
        try:
            return int(y)
        except ValueError:
            return str(y)
    for r in d['results']:
        r['obs'] = len(r['times'])
        r['Ah'] = fn(0)
        r['Aw'] = fn(1)
        r['Bh'] = r['Aw']
        r['Bw'] = fn(2)
        if all([filter_ref, r['Ah'] == 1, r['Aw'] == 1, r['Bw'] == 1]):
            ref = r[col_ref]
            print(f'ref.{col_ref}={ref}')
        _ = r.pop('command')
        _ = r.pop('parameters')

    df = pd.DataFrame(d['results'])
    df['ci'] = 1.960 * df['stddev'] / (df['obs']**0.5)
    df = df[['Ah','Aw','Bh','Bw','obs','min','mean','stddev','ci']]
    df['speedup'] = ref/df[col_ref]
    df = df.sort_values(col_ref)
    if filter_ref:
        df = df[df[col_ref] < ref]

    return df

In [6]:
df = prep(out+'alg2.json', col_ref='mean')[:10]
df.to_markdown(out+'alg2.md', index=False)
df

ref.mean=0.7139999309799999


,Ah,Aw,Bh,Bw,obs,min,mean,stddev,ci,speedup
999,256,4,4,512,10,0.571092,0.584602,0.008179,0.005070,1.221343
878,128,4,4,512,10,0.576974,0.585833,0.006155,0.003815,1.218777
636,32,4,4,512,10,0.576606,0.586720,0.007448,0.004616,1.216935
515,16,4,4,512,10,0.573534,0.587348,0.008694,0.005389,1.215634
757,64,4,4,512,10,0.569514,0.589697,0.009309,0.005770,1.210792
868,128,2,2,1024,10,0.575697,0.591686,0.012119,0.007512,1.206722
989,256,2,2,1024,10,0.580246,0.591964,0.007818,0.004846,1.206153
747,64,2,2,1024,10,0.583426,0.593303,0.010737,0.006655,1.203431
505,16,2,2,1024,10,0.568327,0.593998,0.012405,0.007689,1.202025
1032,256,32,32,512,10,0.578671,0.594083,0.010583,0.006560,1.201851


In [7]:
df = prep(out+'alg2_fine.json', col_ref='mean', filter_ref=False)
df[:10].to_markdown(out+'alg2_fine.md', index=False)
df[:10]

,Ah,Aw,Bh,Bw,obs,min,mean,stddev,ci,speedup
39,160,4,4,512,50,0.567917,0.579511,0.006830,0.001893,1.232074
40,164,4,4,512,50,0.568295,0.580473,0.007377,0.002045,1.230031
46,188,4,4,512,50,0.568068,0.580799,0.007473,0.002071,1.229341
62,252,4,4,512,50,0.568809,0.580831,0.007891,0.002187,1.229273
41,168,4,4,512,50,0.568440,0.580891,0.006163,0.001708,1.229147
17,72,4,4,512,50,0.567241,0.581942,0.006998,0.001940,1.226925
30,124,4,4,512,50,0.568977,0.582859,0.007257,0.002011,1.224997
49,200,4,4,512,50,0.570212,0.582996,0.007625,0.002114,1.224707
59,240,4,4,512,50,0.569755,0.583275,0.007779,0.002156,1.224123
21,88,4,4,512,50,0.567583,0.583355,0.009229,0.002558,1.223954


In [8]:
fig = px.line(df.sort_values(by='Ah'), x='Ah', y='speedup')
fig.update_layout(xaxis={'dtick': 16, 'range':[0,df['Ah'].max()]})
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.write_image(out+'fine.png', scale=2)
fig.show()

In [9]:
df = prep(out+'alg2_best.json', col_ref='mean', filter_ref=False)
df

,Ah,Aw,Bh,Bw,obs,min,mean,stddev,ci,speedup
1,160,4,4,512,100,0.573855,0.583267,0.004306,0.000844,1.224140
0,n,n,n,n,100,0.654047,0.687684,0.017848,0.003498,1.038268


In [10]:
min(df['mean'])/max(df['mean'])-1

-0.1518389550718997

In [11]:
final = pd.read_csv(out+'all.csv')
assert len(final[final['result_check[bool]']==0]) == 0
res = final\
    .groupby(['algorithm[int]','N[int]'])['time[s]']\
    .agg(['count','mean','std'])
res['ci'] = 1.960 * res['std'] / (res['count']**0.5)
res['speedup0'] = 1.0
res['speedup1'] = 1.0
for n in res.index.unique(level=1):
    for a in range(3):
        for b in range(2):
            res.at[(a,n),f'speedup{b}'] = \
                res.at[(b,n),'mean'] / res.at[(a,n),'mean']
res = res.reset_index()
res.insert(0,'N',res.pop('N[int]').astype('string'))
res.insert(0,'algorithm',res.pop('algorithm[int]').astype('string'))
res.to_markdown(out+'res.md', index=False)
res

,algorithm,N,count,mean,std,ci,speedup0,speedup1
0,0,256,10,0.024733,0.000770,0.000477,1.000000,0.406934
1,0,512,10,0.198396,0.002601,0.001612,1.000000,0.397115
2,0,1024,10,6.905933,0.029300,0.018160,1.000000,0.102337
3,0,2048,10,65.023929,0.073947,0.045833,1.000000,0.086912
4,1,256,10,0.010065,0.000578,0.000358,2.457401,1.000000
5,1,512,10,0.078786,0.001883,0.001167,2.518162,1.000000
6,1,1024,10,0.706734,0.023988,0.014868,9.771615,1.000000
7,1,2048,10,5.651341,0.066783,0.041392,11.505929,1.000000
8,2,256,10,0.008985,0.000347,0.000215,2.752638,1.120142
9,2,512,10,0.070187,0.002190,0.001357,2.826663,1.122510


In [12]:
for b in range(2):
    fig = px.bar(res, x='N', y=f'speedup{b}',
        color='algorithm', barmode='group', text_auto=True,
        title=f'Speedup vs. Algoritmo {b}')
    fig.update_yaxes(title='speedup')
    fig.update_traces(textposition='outside')
    fig.update_layout(yaxis_tickformat='.2f',
        margin=dict(l=0, r=0, t=40, b=0),
        legend_x=0,
        legend_bgcolor='rgba(0,0,0,0)')
    fig.write_image(out+f'speedup{b}.png', scale=2)
    fig.show()